In [220]:
import pandas as pd

In [221]:
import numpy as np

In [222]:
from glob import glob

In [223]:
import itertools

In [224]:
import pygsheets as pyg

In [225]:
from apiclient import discovery

In [226]:
from oauth2client.file import Storage

In [227]:
import httplib2

In [228]:
import time

In [402]:
import copy

#### some pandas opts

In [229]:
pd.options.display.max_columns = 100

In [230]:
pd.options.display.max_rows = 500

#### empty dict for storing df that is going to be created below

dfs = {}

#### secret file for auth for anything needed in this notebook

In [231]:
secret_json = "./client_secret_986841527205-ndf6fbsfmrhjinnofm47j6olvmm11smn.apps.googleusercontent.com.json"

#### auth for drive. quick start script that takes secret client json file and creates conctents of
#### ```~/.credentials```
#### is needed!!!
#### it will NOT work inside the notebook since here the argparse Namespace is broken! it is needed for the flags var (see quickstart)

In [232]:
credential_path = "/home/dizak/.credentials/drive-python-quickstart.json"

In [233]:
store = Storage(credential_path)

In [234]:
credentials = store.get()

In [235]:
http = credentials.authorize(httplib2.Http())

In [236]:
service = discovery.build("drive", "v3", http=http)

#### mind if len(results["files"]) > 1000 which is max per page you have to take care of pageToken.
#### pageToken can be retrieved by 
#### ```res = service.files().list(pageSize=1000, fields="nextPageToken, files").execute()```
#### ```token = res["nextPageToken"]```
#### and then passed as arg like:
#### ```service.files().list(pageSize=1000, pageToken=token, fields="nextPageToken, files").execute()```

In [237]:
results = service.files().list(pageSize=1000).execute()

In [238]:
len(results["files"])

508

#### authorize pyg and set retry limit

In [239]:
gc = pyg.authorize(outh_file=secret_json,
                   outh_nonlocal=True,
                   retries=5)

#### title for the google sheet and xls

In [240]:
sheet_name = "BY4741-nup133-day70"

#### xls output directory

In [241]:
xls_out_dir = "/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/BY-nup133-day70/"

#### gdrive output directory

In [242]:
gdrive_out_dir = [i["id"] for i in results["files"] if i["name"] == "testing"][0]

In [243]:
gdrive_out_dir

u'0BwDD9bCCIcilTkMwNGNjWl9vVXc'

#### naming of the sampling levels. it is INDEPENDENT of the actual naming in the CSVs

In [244]:
sampling_levels = ["N1", "N2", "N3"]

#### columns names for by which sorting will be done

In [245]:
sort_vals = ["Minimum"]

#### CSVs paths

In [246]:
strain_gene_day_N1_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/BY-nup133-day70/N1/*csv")

In [247]:
strain_gene_day_N2_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/BY-nup133-day70/N2/*csv")

In [248]:
strain_gene_day_N3_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/BY-nup133-day70/N3/*csv")

#### get non-redundant list of genes in the inputfiles

In [249]:
def find_flat_value(inputfiles_list,
                    col_name = "CDS"):
    """
    Get flat list of desired values from list of CSV files.
    
    Parameters
    -------
    inputfiles_list: list of str
        List of input CSV files.
    col_name: str
        Desired column name in the input CSV file.
    
    Returns
    -------
    list of desired values.
    """
    values_list = []
    for i in inputfiles_list:
        df = pd.read_csv(i)
        if len(df) == 0:
            pass
        elif col_name not in df.columns:
            pass
        else:
            values_list.append(df[col_name].dropna().drop_duplicates().tolist())
    return list(itertools.chain.from_iterable(values_list))

#### get values from input files by key

In [250]:
def find_by_key(inputfiles_list,
                key):
    """
    Get pandas.DataFrame selected by a given key from list of CSV files.
    
    Parameters
    -------
    inputfiles_list: list of str
        List of input CSV files.
    key: str, int, float, bool
        Key used as query against rows in the CSV files.
    value_col: str
        Column name which holds values to be returned.
    
    Returns
    -------
    dict of lists of desired values if pandas.Dataframe not empty
    None if pandas.DataFrame empty
    """
    values_list = []
    for i in inputfiles_list:
        filename = "".join(i.split("/")[-1].split(".")[:-1])
        df = pd.read_csv(i)
        if len(df) == 0:
            pass
        else:
            if isinstance(key, str) == True:
                df_dtype_sel = df.select_dtypes(include=["object"])
            elif isinstance(key, int) == True:
                df_dtype_sel = df.select_dtypes(include=["int"])
            elif isinstance(key, float) == True:
                df_dtype_sel = df.select_dtypes(include=["float"])
            elif isinstance(key, bool) == True:
                df_dtype_sel = df.select_dtypes(include=["bool"])
            else:
                raise ValueError("key must str, int, float or bool dtype")
            for col in df_dtype_sel.columns:
                df_sel = df[df_dtype_sel[col] == key]
                if len(df_sel) > 0:
                    return {"dataframe": df_sel,
                            "filename": filename}

#### get whole set of pandas.DataFrames selections in one dict

In [251]:
def get_dfs_set(key_list,
                files_list,
                vals=["Minimum",
                      "Maximum",
                      "Change"],
                df_key="dataframe",
                key_index_name="Gene",
                smpl_index_name="Sample",
                row_index_name="Number",
                index_by_key=True,
                smpl_index_val=None,):
    """
    Get desired values in pandas.Dataframe gathered in dict by
    the list of keys.
    
    Parameters
    -------
    key_list: list, tuple
        List of keys by which pandas.Dataframes are
        initially selected.
    files_list: list, tuple
        List of input CSV files.
    vals: list of str, default: ["Minimum", 
                                 "Maximum",
                                 "Change"]
        Columns names holding data in pandas.DataFrames
        selected.
    df_key: str, default: <"dataframe">
        Key for generic pandas.DataFrame selection for
        SNPs-sheets_merge.find_by_key function.
    key_index_name: str, default: <"Gene">
        Name for index of selection key.
    smpl_index_name: str, default: <"Sample">
        Name for index of sample.
    row_index_name: str, default: <"Number">
        Name for numeric row index.
    index_by_key: bool, default: True
        Enables multiindexing.
    smpl_index_val: str, default: <None>
        Adds sample name to multiindexing if not <None>
    """
    out_dict = {}
    for i in key_list:
        key_vals = find_by_key(files_list,
                               key=i)[df_key]
        out_dict[i] = key_vals[vals]
    if index_by_key is True:
        for i in out_dict:
            key_index = [i] * len(out_dict[i])
            if smpl_index_val is not None:
                smpl_index = [smpl_index_val] * len(out_dict[i])
                tpls = list(zip(*[key_index,
                                  smpl_index,
                                  out_dict[i].index]))
                mindex = pd.MultiIndex.from_tuples(tpls,
                                                   names=[key_index_name,
                                                          smpl_index_name,
                                                          row_index_name])
            else:
                tpls = list(zip(*[key_index,
                                  out_dict[i].index]))
                mindex = pd.MultiIndex.from_tuples(tpls,
                                                   names=[key_index_name,
                                                          row_index_name])
            out_dict[i].index = mindex
    return out_dict

#### merge any given number of dfs

In [252]:
def merge_dfs(dfs,
              sort_cols=["Minimum",
                         "Maximum"],
              reconstr_index=True):
    """
    Merge any number of pandas.DataFrame into one.
    Indexes must be identical in all the pandas.DataFrames.
    
    Parameters
    -------
    dfs: list
        list of pandas.DataFrames to merge.
    sort_cols: list, None
        list of col names to sort the final
        pandas.DataFrame by. No sorting if None.
    reconstr_index: bool
        Copy index from the original pandas.DataFrames
        to the final one.
    """
    for x in [set(i.index) for i in dfs]:
        assert len(x) == 1, "Indices are not homogenic."
    new_index = list(x)
    df = reduce(lambda df1, df2: pd.merge(left=df1,
                                          right=df2,
                                          how="outer"),
                dfs)
    if sort_cols is not None:
        df.sort(columns=sort_cols)
    if reconstr_index is True:
        df.index = len(df) * new_index
    return df

#### let's find out which CDS are present in all the files one by one

In [253]:
strain_gene_day_N1_CDSs = find_flat_value(strain_gene_day_N1_files)
strain_gene_day_N2_CDSs = find_flat_value(strain_gene_day_N2_files)
strain_gene_day_N3_CDSs = find_flat_value(strain_gene_day_N3_files)

#### let's gather some info about each of the CDS from each sample

In [254]:
des_cols = ["Change",
            "Protein Effect",
            "Polymorphism Type",
            "Variant Frequency",
            "Average Quality",
            "Minimum",
            "Maximum"]

In [255]:
strain_gene_day_N1_dfs = get_dfs_set(strain_gene_day_N1_CDSs,
                                     strain_gene_day_N1_files,
                                     vals=des_cols,
                                     smpl_index_val=sampling_levels[0])
strain_gene_day_N2_dfs = get_dfs_set(strain_gene_day_N2_CDSs,
                                     strain_gene_day_N2_files,
                                     vals=des_cols,
                                     smpl_index_val=sampling_levels[1])
strain_gene_day_N3_dfs = get_dfs_set(strain_gene_day_N3_CDSs,
                                     strain_gene_day_N3_files,
                                     vals=des_cols,
                                     smpl_index_val=sampling_levels[2])

#### let's unwind them all from this dict into lists

In [256]:
N1_dfs = [strain_gene_day_N1_dfs[i] for i in strain_gene_day_N1_dfs.keys()]
N2_dfs = [strain_gene_day_N2_dfs[i] for i in strain_gene_day_N2_dfs.keys()]
N3_dfs = [strain_gene_day_N3_dfs[i] for i in strain_gene_day_N3_dfs.keys()]

#### let's merge them to see changes between samples.
#### rememeber now cannot use the merge_dfs function since multindex is NOT homogenic

In [257]:
N1_df = reduce(lambda df1, df2: pd.concat([df1, df2]),
               N1_dfs)

In [258]:
N2_df = reduce(lambda df1, df2: pd.concat([df1, df2]),
               N2_dfs)

In [259]:
N3_df = reduce(lambda df1, df2: pd.concat([df1, df2]),
               N3_dfs)

In [260]:
N1_N2_N3_df = reduce(lambda df1, df2: pd.concat([df1, df2]),
                     [N1_df, N2_df, N3_df])

#### now let's clean this messy Variant Frequency col

In [261]:
N1_N2_N3_df["Variant Frequency"] = N1_N2_N3_df["Variant Frequency"].replace("%", "", regex=True).\
                                                                    replace(".+->", "", regex=True).\
                                                                    astype("float")

#### and sort everything by the Minimum value so that repeatitions can be easily spotted

In [262]:
N1_N2_N3_df.sort_values(sort_vals, inplace=True)

#### let's make xls

writer = pd.ExcelWriter("{}/{}.xls".format(xls_out_dir, sheet_name))

for i in set(strain_gene_day_N1_CDSs +
             strain_gene_day_N2_CDSs +
             strain_gene_day_N3_CDSs):
    N1_N2_N3_df.xs(i).to_excel(excel_writer=writer,
                               sheet_name=i)

writer.save()

#### let's make sheet on gdrive

sh = gc.create(sheet_name, parent_id=gdrive_out_dir)

for i in set(strain_gene_day_N1_CDSs +
             strain_gene_day_N2_CDSs +
             strain_gene_day_N3_CDSs):
    time.sleep(3)
    wks = sh.add_worksheet(i)
    wks.set_dataframe(N1_N2_N3_df.xs(i),
                      (1, 1),
                      copy_index=True,
                      fit=True)
sh.del_worksheet(sh.worksheet_by_title("Sheet1"))

## optional appending DataFrame to list,
## so that it can be used for merging everything into one, unintelligible mess...

In [263]:
dfs[sheet_name] = N1_N2_N3_df

In [264]:
len(dfs)

5

# this part of the notebook should be treated separately!
## it is ment to be used for merging ALL previusly created sheets into ONE
## since it is so, vars needed for this purpose are defined here instead of being on the TOP

#### let's add key of each DataFrame - which is its strain-deletion-day name - as column

In [287]:
for i in dfs.keys():
    dfs[i]["Strain"] = i

#### let's reset indices in each DataFrame to regular columns and get rid of the Number col

In [288]:
for i in dfs.keys():
    dfs[i].reset_index(level=[0, 1, 2], inplace=True).drop(columns="Number", inplace=True)

#### columns on which merge will take place

In [380]:
merge_cols = ["Gene",
              "Change",
              "Protein Effect",
              "Polymorphism Type",
              "Minimum",
              "Maximum"]

#### columns which should be properly suffixed

In [381]:
specific_cols = [i for i in dfs.values()[0] if i not in merge_cols]

#### reduce takes it nicely but suffixes are pain
#### so let's try with reduce but after renaming columns instead of suffixing them

In [394]:
for frame in dfs.values():
    for i in specific_cols:
        frame.rename(columns={i: "{} {}".format(i, frame["Strain"][0])}, inplace=True)

In [396]:
df_m = reduce(lambda df1, df2: pd.merge(left=df1,
                                        right=df2,
                                        on=merge_cols,
                                        how="outer"),
              dfs.values())

#### copy df_m b4 breaking it...

In [404]:
df_m_copy = copy.deepcopy(df_m)

## some cleaning
#### remove cols that consist only of NaN

In [409]:
df_m.dropna(axis=1, how="all", inplace=True)

#### no need for *Strain* columns now

In [405]:
undes_cols = [i for i in df_m.columns if "Strain" in i]

In [426]:
df_m.drop(columns=undes_cols, inplace=True)

#### it is all fine but there is some mess in the columns order...

In [432]:
specific_cols_dict = {}
for col_group in specific_cols:
    specific_cols_dict[col_group] = [i for i in df_m.columns if col_group in i]

In [434]:
del specific_cols_dict["Strain"]

In [441]:
ordered_cols = list(itertools.chain.from_iterable(specific_cols_dict.values()))

In [442]:
ordered_cols

['Sample W303-nup133-day42',
 'Sample BY4741-nup133-day70',
 'Sample W303-cog7-day42',
 'Sample BY4741-WT-day70',
 'Sample W303-WT-day42',
 'Variant Frequency W303-nup133-day42',
 'Variant Frequency BY4741-nup133-day70',
 'Variant Frequency W303-cog7-day42',
 'Variant Frequency BY4741-WT-day70',
 'Variant Frequency W303-WT-day42',
 'Average Quality W303-nup133-day42',
 'Average Quality BY4741-nup133-day70',
 'Average Quality W303-cog7-day42',
 'Average Quality BY4741-WT-day70',
 'Average Quality W303-WT-day42']

In [445]:
df_m_ord = df_m[merge_cols + ordered_cols]

In [446]:
df_m_ord

,Gene,Change,Protein Effect,Polymorphism Type,Minimum,Maximum,Sample W303-nup133-day42,Sample BY4741-nup133-day70,Sample W303-cog7-day42,Sample BY4741-WT-day70,Sample W303-WT-day42,Variant Frequency W303-nup133-day42,Variant Frequency BY4741-nup133-day70,Variant Frequency W303-cog7-day42,Variant Frequency BY4741-WT-day70,Variant Frequency W303-WT-day42,Average Quality W303-nup133-day42,Average Quality BY4741-nup133-day70,Average Quality W303-cog7-day42,Average Quality BY4741-WT-day70,Average Quality W303-WT-day42
0,COS7 CDS,T -> A,Substitution,SNP (transversion),1933,1933,N2,NaN,N2,NaN,N1,96.3,NaN,71.9,NaN,96.7,27.0,NaN,29.0,NaN,28.0
1,COS7 CDS,T -> A,Substitution,SNP (transversion),1933,1933,N2,NaN,N2,NaN,N3,96.3,NaN,71.9,NaN,79.6,27.0,NaN,29.0,NaN,27.0
2,COS7 CDS,T -> A,Substitution,SNP (transversion),1933,1933,N2,NaN,N2,NaN,N2,96.3,NaN,71.9,NaN,83.1,27.0,NaN,29.0,NaN,31.0
3,COS7 CDS,T -> A,Substitution,SNP (transversion),1933,1933,N2,NaN,N1,NaN,N1,96.3,NaN,76.4,NaN,96.7,27.0,NaN,30.0,NaN,28.0
4,COS7 CDS,T -> A,Substitution,SNP (transversion),1933,1933,N2,NaN,N1,NaN,N3,96.3,NaN,76.4,NaN,79.6,27.0,NaN,30.0,NaN,27.0
5,COS7 CDS,T -> A,Substitution,SNP (transversion),1933,1933,N2,NaN,N1,NaN,N2,96.3,NaN,76.4,NaN,83.1,27.0,NaN,30.0,NaN,31.0
6,COS7 CDS,T -> A,Substitution,SNP (transversion),1933,1933,N1,NaN,N2,NaN,N1,90.9,NaN,71.9,NaN,96.7,28.0,NaN,29.0,NaN,28.0
7,COS7 CDS,T -> A,Substitution,SNP (transversion),1933,1933,N1,NaN,N2,NaN,N3,90.9,NaN,71.9,NaN,79.6,28.0,NaN,29.0,NaN,27.0
8,COS7 CDS,T -> A,Substitution,SNP (transversion),1933,1933,N1,NaN,N2,NaN,N2,90.9,NaN,71.9,NaN,83.1,28.0,NaN,29.0,NaN,31.0
9,COS7 CDS,T -> A,Substitution,SNP (transversion),1933,1933,N1,NaN,N1,NaN,N1,90.9,NaN,76.4,NaN,96.7,28.0,NaN,30.0,NaN,28.0


#### let's make sheet on gdrive

In [450]:
sh = gc.create("all_merged", parent_id=gdrive_out_dir)

In [451]:
wks=sh.add_worksheet("all_merged")
wks.set_dataframe(df_m_ord,
                  (1, 1),
                  copy_index=True,
                  fit=True)
sh.del_worksheet(sh.worksheet_by_title("Sheet1"))

#### experiments with squash

In [475]:
cols = ["Gene",
        "Change",
        "Protein Effect",
        "Polymorphism Type",
        "Minimum",
        "Maximum"] + [i for i in df_m_ord.columns if "Sample" in i]

In [478]:
[i for i in df_m_ord.columns if "Sample" in i]

['Sample W303-nup133-day42',
 'Sample BY4741-nup133-day70',
 'Sample W303-cog7-day42',
 'Sample BY4741-WT-day70',
 'Sample W303-WT-day42']

In [483]:
df_m_small = df_m_ord[cols]

In [536]:
df_m_small_cos = df_m_small[df_m_small.Gene == "COS7 CDS"]

In [587]:
df_test = df_m_small_cos.groupby("Change").apply(lambda x: "-".join(set(x["Sample W303-nup133-day42"].dropna().drop_duplicates())))

In [601]:
df_test2 = df_m_small_cos.groupby("Change").apply(lambda x: "-".join(set(x["Sample W303-cog7-day42"].dropna().drop_duplicates())))

In [602]:
df_test3 = df_m_small_cos.groupby("Change").apply(lambda x: "-".join(set(x["Sample W303-WT-day42"].dropna().drop_duplicates())))

In [609]:
pd.merge(left=, right=pd.DataFrame([df_test, df_test2, df_test3]).T)

,0,1,2
Change,,,
+C,N1-N3,,
-A,N1-N3,,
A -> C,N1-N2-N3,N1-N2,N1-N2-N3
A -> G,N1-N2-N3,N1-N2,N1-N2-N3
AAT -> TAC,N1-N2-N3,,
AT -> CA,N1-N2-N3,,N1
AT -> GC,N1-N2,,
ATC -> GGA,N2,,
C -> A,N1-N2-N3,N1,N1-N2-N3
